# Catering Company - Take Home Exercise

### 1. determine the total dollar amount of coffee sales on our platform
### 2. suggest an approach for a coffee-pairing recommender (i.e. they've add coffee, what can/should we suggest they add?)

By - Jarred Bultema

In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.sparse import hstack


In [148]:
## import data
data_path = '../data/metro-coffee-orders.csv'
df = pd.read_csv(data_path, index_col=None)
df.head()

,ORDER_ID,CATERER_ID,ITEM_ID,ITEM_NAME,ITEM_DESCRIPTION,ITEM_PRICE,ITEM_QUANTITY
0,214561246,129332,5352507,Pilgrim Burger,"With a turkey patty, stuffing, American cheese...",14.5,1
1,212301212,129332,5352507,Pilgrim Burger,"With a turkey patty, stuffing, American cheese...",14.5,1
2,215326277,129332,5352507,Pilgrim Burger,"With a turkey patty, stuffing, American cheese...",14.5,1
3,214561246,129332,5352507,Pilgrim Burger,"With a turkey patty, stuffing, American cheese...",14.5,1
4,214561246,129332,5352504,Burger Bomb,"With a beef patty, sauteed peppers, mushroom, ...",15.0,1


In [149]:
## quick look at dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44774 entries, 0 to 44773
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDER_ID          44774 non-null  int64  
 1   CATERER_ID        44774 non-null  int64  
 2   ITEM_ID           44774 non-null  int64  
 3   ITEM_NAME         44774 non-null  object 
 4   ITEM_DESCRIPTION  39233 non-null  object 
 5   ITEM_PRICE        44774 non-null  float64
 6   ITEM_QUANTITY     44774 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 2.4+ MB


In [150]:
## missing values in item-description...
df[df['ITEM_DESCRIPTION'].isna()].describe()

,ORDER_ID,CATERER_ID,ITEM_ID,ITEM_PRICE,ITEM_QUANTITY
count,5.541000e+03,5541.000000,5.541000e+03,5541.000000,5541.000000
mean,2.139519e+08,203976.144378,5.057343e+06,14.379709,4.267100
std,1.363028e+06,86418.340539,1.970265e+06,23.003039,9.645211
min,2.043744e+08,1136.000000,8.113000e+03,0.050000,1.000000
25%,2.130177e+08,129160.000000,3.139203e+06,2.750000,1.000000
50%,2.139568e+08,227856.000000,5.619153e+06,4.500000,1.000000
75%,2.149881e+08,266885.000000,6.749527e+06,14.500000,3.000000
max,2.166096e+08,338933.000000,7.510281e+06,352.000000,185.000000


In [151]:
df[df['ITEM_DESCRIPTION'].isna()]['ITEM_NAME'].value_counts().head(25)

ITEM_NAME
Bottled Water                                211
Cheese Pizza                                 206
Chocolate Chip Cookie                        161
Assorted Individual Sodas                    132
Utensils                                      97
Box of Coffee                                 77
Pepperoni Pizza                               67
Fruit Platter                                 46
Brownie                                       44
Sweet Potato Fries                            42
Basmati Rice                                  36
Chips & Guacamole                             35
Garden Salad                                  34
Coke                                          32
Cookie                                        32
Chocolate Chip Cookies                        31
Diet Coke                                     31
Pita Bread                                    30
Plantain Chips                                29
Fruit Bowl                                    29
Spicy Tuna

## Create the Naive target analysis (VO)

In [152]:
# no issue here, standard business logic
df['item_total'] = df['ITEM_PRICE'] * df['ITEM_QUANTITY']
####### START REVIEW CODE BLOCK #########
# help me review this line of code
df['is_coffee'] = df['ITEM_NAME'].str.lower().str.contains('coffee')
####### END REVIEW CODE BLOCK #########
# no issue here, but suggestions welcome
total_with_coffee = df[df['is_coffee']]['item_total'].sum().round(2)
total_without_coffee = df[~df['is_coffee']]['item_total'].sum().round(2)
coffee_percentage = round((total_with_coffee / total_without_coffee) * 100, 2)
print(f'Coffee total V0: ${total_with_coffee:,.2f}')
print(f'Non-Coffee total V0: ${total_without_coffee:,.2f}')
print(f'Coffee sales percentage V0: {coffee_percentage}%')


Coffee total V0: $10,068.42
Non-Coffee total V0: $1,509,257.92
Coffee sales percentage V0: 0.67%


In [153]:
## look a bit more on the naive target label
df['is_coffee'].value_counts()


is_coffee
False    44523
True       251
Name: count, dtype: int64

In [154]:
## check the items detected as coffee
df[df['is_coffee']]['ITEM_NAME'].value_counts()

ITEM_NAME
Box of Coffee                            113
Fresh Brewed Stumptown Coffee             27
Coffee                                    20
Box of Pierce Bros. Coffee                13
Dark Roast Coffee                         10
Coffee Service for 10                      8
Coffee Lover                               8
Coffee Bowl                                7
Box O' Joe (Hot Coffee)                    7
Fresh Brewed Coffee                        6
Brew for the Crew Coffee                   6
The Dunkin' Continental w/ Hot Coffee      3
Coffee Cake Tray                           3
Iced Coffee                                3
Joe to Go Coffee Box                       2
Cold Brew Coffee                           2
Coffee Tote                                2
Regular Coffee                             2
Cinnamon Coffee Cake                       1
Freshly-Brewed Coffee                      1
96 Oz Box of Coffee                        1
Deluxe Coffee & Bagel Blend                1


In [155]:
## check the item descriptions for these coffee items
df[df['is_coffee']][['ITEM_NAME', 'ITEM_DESCRIPTION']].drop_duplicates()

,ITEM_NAME,ITEM_DESCRIPTION
802,Brew for the Crew Coffee,"96 oz fresh-brewed and hot coffee, ready to en..."
1561,Box of Coffee,NaN
1827,Coffee Bowl,"Blended coffee, almond butter, bananas, cashew..."
1896,Coffee Lover,"With almond milk, coffee, almond butter, banan..."
2427,Thai Iced Coffee,NaN
3111,Iced Coffee,With your choice of milk.
4151,Box O' Joe (Hot Coffee),Hot Coffee for a crowd! Dunkin’ Box O’ Joe con...
4157,The Dunkin' Continental w/ Hot Coffee,"This classic spread comes with hot coffee, ora..."
5329,Box of Coffee,With your choice of coffee.
5358,Joe to Go Coffee Box,Each Joe to Go contains enough coffee for 12 e...


#### There are least a few examples of non-coffee items detected by this naive approach.

#### Are there also coffee items that are missed?


In [156]:
df[(df['is_coffee']==False) & (df['ITEM_DESCRIPTION'].str.contains('cof'))][['ITEM_NAME', 'ITEM_DESCRIPTION']].drop_duplicates()

,ITEM_NAME,ITEM_DESCRIPTION
1554,Finagle's Egg Sandwich Platter,"With egg, cheese, and your choice of topping o..."
1560,Happy Bagel Breakfast,"With assorted bagels, assorted whipped cream c..."
1567,Rise & Shine Breakfast,"With assorted bagels, loaf cakes, scones, muff..."
4770,The Early Bird,Assorted breakfast pastries. Served with coffee.
6863,The Boardroom Package,An assortment of mini breakfast pastries with ...
7991,Energize Smoothie,"With coffee, peanut butter, banana, cacao, and..."
8387,Juicygreens Rise and Shine Continental Breakfa...,"Includes coffee, orange juice, fruit cups, car..."
9931,The Continental Breakfast,"Includes mini muffins, mini Danish, coffee cak..."
9938,Executive Breakfast Buffet,"Includes your choice of breakfast meat, scramb..."
10582,Breakfast Pastry Tray,"Includes assorted bagels, scones, coffee cakes..."


In [157]:
## Naive "coffee" items
## there are few enough that I can review
df[df['is_coffee']]['ITEM_NAME'].nunique(),\
df[df['is_coffee']]['ITEM_DESCRIPTION'].nunique()

(27, 22)

In [158]:
## too many names and descriptions for manual review...
df[~df['is_coffee']]['ITEM_NAME'].nunique(),\
df[~df['is_coffee']]['ITEM_DESCRIPTION'].nunique()

(7266, 7354)

### Refined Target-approach 1 - Use a list of keywords rather than just single term

This approach employs a simple user-defined list of coffee-equivalent words to define the target. This still has potential to (1) miss coffee-sales when a different keyword is used and (2) include non-coffee items that might have these terms (theoretical ex. "chocolate covered espresso beans")

Given the number of unique names and descriptions, it would be best to use a scalable NLP-approach rather than manual review. But, given the time constraints I'll quickly check for positive-associated terms.

In [159]:
## EDA to determine potential coffee-synonym terms
# focus on the ITEM_NAME column first
item_names = df['ITEM_NAME']

# convert to lowercase and remove punctuation
item_names = item_names.str.lower().apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# assuming 'is_coffee' column is accurate and exists
coffee_items = item_names[df['is_coffee']]

# tokenize the text and count word frequency
word_list = ' '.join(coffee_items).split()
word_counts = Counter(word_list)

# display the most common words in coffee item names
common_coffee_words = pd.DataFrame(word_counts.most_common(20), columns=['term', 'count'])
common_coffee_words

,term,count
0,coffee,251
1,box,137
2,of,127
3,fresh,33
4,brewed,33
5,stumptown,27
6,for,14
7,pierce,13
8,bros,13
9,hot,10


In [160]:
## Repeat this approach with the item-descriptions too
item_desc = df['ITEM_DESCRIPTION'].fillna('')
item_desc = item_desc.str.lower().apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
coffee_desc = item_desc[df['is_coffee']]

# tokenize the text and count word frequency
word_list_desc = ' '.join(coffee_desc).split()
word_counts_desc = Counter(word_list_desc)

# display the most common words
common_coffee_desc_words = pd.DataFrame(word_counts_desc.most_common(20), columns=['term', 'count'])
common_coffee_desc_words

,term,count
0,coffee,129
1,of,94
2,your,89
3,choice,82
4,and,72
5,with,68
6,milk,56
7,half,54
8,served,52
9,roast,45


In [161]:
## let's briefly manually review these keywords to see if they seem good
coffee_keywords = ['coffee', 'espresso', 'latte', 'cappuccino', 'mocha', 'americano', 'macchiato', 'brewed', 'brew']

for term in coffee_keywords:
    tmp_desc = df[df['ITEM_DESCRIPTION'].fillna('').str.contains(term)]['ITEM_NAME'].unique().tolist()
    tmp_name = df[df['ITEM_NAME'].str.contains(term)]['ITEM_NAME'].unique().tolist()

    print(term, tmp_desc, '\n', tmp_name, '\n')

coffee ['Brew for the Crew Coffee', "Finagle's Egg Sandwich Platter", 'Happy Bagel Breakfast', 'Rise & Shine Breakfast', 'Coffee Bowl', 'Coffee Lover', "The Dunkin' Continental w/ Hot Coffee", 'The Early Bird', 'Box of Coffee', 'Joe to Go Coffee Box', 'The Boardroom Package', 'Energize Smoothie', 'Juicygreens Rise and Shine Continental Breakfast Menu', 'The Continental Breakfast', 'Executive Breakfast Buffet', 'Breakfast Pastry Tray', 'Super Breakfast', 'Freshly-Brewed Coffee', 'Super Hot Breakfast Buffet', 'Continental Breakfast', 'Healthy Breakfast', 'Premium Breakfast', 'Basic Breakfast', 'Winner Takes All Breakfast', 'Deluxe Coffee & Bagel Blend', 'Coffee', 'Mini Pastries & Muffins', 'Flavored Coffee Cake', 'Employee Appreciation Month Breakfast', 'Mocha Madness Smoothie', 'Signature Breakfast Package', 'Breakfast Nosh', 'Coffee Cake Tray', 'Assorted Cupcakes', 'Coffee Tote', 'Fresh Brewed Stumptown Coffee', 'The Junior Continental Breakfast', 'VIP Hot Breakfast', 'Healthy Start Br

In [162]:
## some terms seem coffee related, but are too general and would get non-coffee items
df[df['ITEM_DESCRIPTION'].fillna('').str.contains('cups')]['ITEM_NAME'].unique()

array(["Box O' Joe (Hot Coffee)", "The Dunkin' Continental w/ Hot Coffee",
       'Joe to Go Coffee Box', 'Seasonal Lemonade Pitcher',
       'Hibiscus Iced Tea Pitcher',
       'Juicygreens Rise and Shine Continental Breakfast Menu',
       'Box of Coffee', 'Freshly-Brewed Coffee', 'Java Box', 'Fruit Cups',
       'Assorted Italian Pastries', 'Breakfast Pastries',
       'Fresh Brewed Stumptown Coffee', 'Assorted Mem Tea',
       'Strawberry Lemonade', 'Lemonade', 'Iced Tea',
       'Broccoli Cheddar Soup'], dtype=object)

In [163]:
# Expanding the criteria to include more coffee-related terms
# We define a list of keywords that are likely to be in the names of coffee items
coffee_keywords = ['coffee', 'espresso', 'latte', 'cappuccino', 'mocha', 'americano', 'macchiato', 'roast', 'brewed', 'brew', 'joe']

# Creating a new column 'is_coffee_v1' with expanded criteria
df['is_coffee_v1'] = df['ITEM_NAME'].str.lower().apply(lambda x: any(keyword in x for keyword in coffee_keywords))

# Now recalculating the total dollar amount of coffee sales
total_with_coffee_v1 = df[df['is_coffee_v1']]['item_total'].sum().round(2)
total_without_coffee_v1 = df[~df['is_coffee_v1']]['item_total'].sum().round(2)
coffee_percentage_v1 = round((total_with_coffee_v1 / total_without_coffee_v1) * 100, 2)

print(f'Coffee total V1: ${total_with_coffee_v1:,.2f}')
print(f'Non-Coffee total V1: ${total_without_coffee_v1:,.2f}')
print(f'Coffee sales percentage V1: {coffee_percentage_v1}%')


Coffee total V1: $136,588.27
Non-Coffee total V1: $1,382,738.07
Coffee sales percentage V1: 9.88%


### Refined Target-approach 2 - Use a list of positive-associated and negative-associated terms

This approach employs a simple user-defined list of coffee-equivalent words to define the target. This still has potential to (1) miss coffee-sales when a different keyword is used and (2) include non-coffee items that might have these terms (theoretical ex. "chocolate covered espresso beans")

Given the number of unique names and descriptions, it would be best to use a scalable NLP-approach rather than manual review. But, given the time constraints I'll quickly check for positive-associated terms and check for negative-associated terms. This will create inclusion-terms and exclusion-terms to use for label assignment

In [164]:
## can we find a simple heuristic to avoid these non-coffee items?
## let's try an exclusion list based on the bad-hits we see
exclusion_keywords = [
    'tea', 'kombucha', 'matcha', 
    'sandwich', 'bagel', 'bread', 'burger', 'sub', 'salad', 'grain', 
    'smoothie', 'tiramisu', 'cake', 'waffle', 'muffin', 
    'vegan', 
    'combo', 'platter', 'tray', 'package', 'catering', 'plate',
    'breakfast', 'lunch', 'continental', 'buffet'
    ]

# Apply lowercase to ensure consistent string matching
df['ITEM_NAME_lower'] = df['ITEM_NAME'].str.lower()
df['ITEM_DESC_lower'] = df['ITEM_DESCRIPTION'].str.lower().fillna('')

for term in coffee_keywords:
    # Filter based on coffee keywords and exclusion keywords
    tmp_desc = df[df['ITEM_DESC_lower'].str.contains(term)]['ITEM_NAME_lower'].unique()
    tmp_name = df[df['ITEM_NAME_lower'].str.contains(term)]['ITEM_NAME_lower'].unique()
    
    # Apply the exclusion criteria
    tmp_desc_clean = [name for name in tmp_desc if all(ex_keyword not in name for ex_keyword in exclusion_keywords)]
    tmp_name_clean = [name for name in tmp_name if all(ex_keyword not in name for ex_keyword in exclusion_keywords)]
    
    print(term, tmp_desc_clean, '\n', tmp_name_clean, '\n')

coffee ['brew for the crew coffee', 'coffee bowl', 'coffee lover', "box o' joe (hot coffee)", 'the early bird', 'box of coffee', 'joe to go coffee box', 'freshly-brewed coffee', 'java box', 'coffee', 'coffee tote', 'fresh brewed stumptown coffee'] 
 ['brew for the crew coffee', 'box of coffee', 'coffee bowl', 'coffee lover', 'thai iced coffee', 'iced coffee', "box o' joe (hot coffee)", 'joe to go coffee box', 'dark roast coffee', 'coffee', 'box of pierce bros. coffee', 'coffee service for 10', 'freshly-brewed coffee', '96 oz box of coffee', 'cold brew coffee', 'fresh brewed coffee', 'coffee tote', 'fresh brewed stumptown coffee', 'iced thai coffee', 'iced milk thai coffee', 'joe-to-go coffee box', 'regular coffee'] 

espresso ['frozen mochaccino'] 
 [] 

latte ['iced craft flavored latte'] 
 ['latte', 'nitro cold brew oat milk latte', 'iced craft flavored latte', 'iced golden milk latte'] 

cappuccino [] 
 ['cappuccino'] 

mocha ['iced craft flavored latte'] 
 ['frozen mochaccino'] 

a

In [165]:
# Expanding the criteria to include more coffee-related terms
# We define a list of keywords that are likely to be in the names of coffee items
coffee_keywords = ['coffee', 'espresso', 'latte', 'cappuccino', 'mocha', 'americano', 'macchiato', 'roast', 'brewed', 'brew', 'joe']

# Exclusion keywords
exclusion_keywords = [
    'tea', 'kombucha', 'matcha', 
    'sandwich', 'bagel', 'bread', 'burger', 'sub', 'salad', 'grain', 
    'smoothie', 'tiramisu', 'cake', 'waffle', 'muffin', 
    'vegan', 
    'combo', 'platter', 'tray', 'package', 'catering', 'plate',
    'breakfast', 'lunch', 'continental', 'buffet'
]

# Convert both ITEM_NAME and ITEM_DESCRIPTION to lowercase for consistent matching
df['ITEM_NAME_lower'] = df['ITEM_NAME'].str.lower()
df['ITEM_DESC_lower'] = df['ITEM_DESCRIPTION'].str.lower().fillna('')  # Fill NaN with empty string

# Define 'is_coffee_v1' based on both ITEM_NAME and ITEM_DESCRIPTION
df['is_coffee_v2'] = df.apply(lambda x: (any(keyword in x['ITEM_NAME_lower'] for keyword in coffee_keywords) or 
                                         any(keyword in x['ITEM_DESC_lower'] for keyword in coffee_keywords)) and 
                                         not any(ex_keyword in x['ITEM_NAME_lower'] for ex_keyword in exclusion_keywords) and 
                                         not any(ex_keyword in x['ITEM_DESC_lower'] for ex_keyword in exclusion_keywords), axis=1)

# Recalculate the total dollar amount of coffee sales
total_with_coffee_v2 = df[df['is_coffee_v2']]['item_total'].sum().round(2)
total_without_coffee_v2 = df[~df['is_coffee_v2']]['item_total'].sum().round(2)
coffee_percentage_v2 = round((total_with_coffee_v2 / total_without_coffee_v2) * 100, 2)

print(f"Coffee total V2: ${total_with_coffee_v2:,.2f}")
print(f"Non-Coffee total V2: ${total_without_coffee_v2:,.2f}")
print(f"Coffee sales percentage V2: {coffee_percentage_v2}%")


Coffee total V2: $49,478.16
Non-Coffee total V2: $1,469,848.18
Coffee sales percentage V2: 3.37%


## Refined Target-approach 3 - NLP modeling

In practice, the challenge with this situation is that we lack extremely accurate target-labels for coffee vs non-coffee items (or non-exclusive coffee items like breakfast platters that have coffee included). The more robust solution is to use NLP models.

I'll demonstrate one such supervised-learning approach, using the labels from V2 approach and assume they're good (mainly just for demonstration purposes).

This approach will use a TFIDF transformation and then build a simple classifier to predict labels. We can also include some of the other numeric features that might be informative.

In [166]:
X_train_tfidf

<35819x3870 sparse matrix of type '<class 'numpy.float64'>'
	with 418402 stored elements in Compressed Sparse Row format>

In [167]:
# assuming 'is_coffee_v2' is an accurate target column
# preparing features
X_text = df['ITEM_NAME'] + ' ' + df['ITEM_DESCRIPTION'].fillna('')  # Combining name and description
X_numerical = df[['ITEM_PRICE', 'ITEM_QUANTITY']]  # Numerical features
# target variable
y = df['is_coffee_v2']

# Splitting the dataset
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(X_text, X_numerical, y, test_size=0.2, random_state=42)

# Text vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train_text)
X_test_tfidf = vectorizer.transform(X_test_text)

# Combine text and numerical features
X_train_combined = hstack([X_train_tfidf, X_train_num])
X_test_combined = hstack([X_test_tfidf, X_test_num])

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train_combined, y_train)

# Model evaluation
predictions = model.predict(X_test_combined)
print(classification_report(y_test, predictions))

# Generate full predictions
X_full_tfidf = vectorizer.transform(X_text)
X_full_combined = hstack([X_full_tfidf, X_numerical])
full_predictions = model.predict(X_full_combined)
df['is_coffee_v3'] = full_predictions

              precision    recall  f1-score   support

       False       1.00      1.00      1.00      8525
        True       1.00      0.94      0.97       430

    accuracy                           1.00      8955
   macro avg       1.00      0.97      0.98      8955
weighted avg       1.00      1.00      1.00      8955



Precision: The ratio of correctly predicted positive observations to the total predicted positives.
- High precision indicates that the model is effective in identifying coffee items, with fewer false positives

Recall: The ratio of correctly predicted positive observations to all observations in the actual class.
- High recall means the model is good at capturing a high proportion of actual coffee items, with fewer false negatives

F1-Score: The weighted average of Precision and Recall. T
- An excellent measure when the class distribution is imbalanced, as it maintains balance of precision and recall.

Support: The number of actual occurrences of the class in the specified dataset.


In [168]:
# Recalculate the total dollar amount of coffee sales
total_with_coffee_v3 = df[df['is_coffee_v3']]['item_total'].sum().round(2)
total_without_coffee_v3 = df[~df['is_coffee_v3']]['item_total'].sum().round(2)
coffee_percentage_v3 = round((total_with_coffee_v3 / total_without_coffee_v3) * 100, 2)

print(f"Coffee total V3: ${total_with_coffee_v3:,.2f}")
print(f"Non-Coffee total V3: ${total_without_coffee_v3:,.2f}")
print(f"Coffee sales percentage V3: {coffee_percentage_v3}%")

# VS V2 approach which created the target labels
# I expect similar numbers
print(f"\nCoffee total V2: ${total_with_coffee_v2:,.2f}")
print(f"Non-Coffee total V2: ${total_without_coffee_v2:,.2f}")
print(f"Coffee sales percentage V2: {coffee_percentage_v2}%")


Coffee total V3: $48,334.99
Non-Coffee total V3: $1,470,991.35
Coffee sales percentage V3: 3.29%

Coffee total V2: $49,478.16
Non-Coffee total V2: $1,469,848.18
Coffee sales percentage V2: 3.37%


## Upsell Feature based on item-pairings

- Determine item pairings that are often sold with coffee and perform frequency analysis
- We'd ideally have more customer, item-category, and context info (time, date, etc) to make this analysis better

In [169]:
# create proxy-category feature for breakfast vs lunch to segment this analysis
# this relies on a prior assumption that coffee is more likely to be included with breakfast orders rather than lunch/dinner

# Define your category keywords
breakfast_keywords = ['egg', 'pancake', 'muffin', 'bagel', 'breakfast', 
                      'oatmeal', 'granola', 'yoghurt', 'cereal', 'waffle', 'continental', 'smoothie']
lunch_keywords = ['combo', 'sandwich', 'salad', 'burger', 'wrap', 'taco', 'pizza', 'veggie', 'vegan']

def categorize_item(item_name):
    item_name = item_name.lower()
    if any(keyword in item_name for keyword in lunch_keywords):
        # put Lunch before Breakfast so it captures those obvious lunch items first
        return 'Lunch'
    elif any(keyword in item_name for keyword in breakfast_keywords):
        return 'Breakfast'
    else:
        return 'Other'

# Apply the function to create a new category column
df['Category'] = df['ITEM_NAME'].apply(categorize_item)
df['Category'].value_counts()

Category
Other        28167
Lunch        15318
Breakfast     1289
Name: count, dtype: int64

In [170]:
# analyze coffee pairings based on these categories
coffee_orders = df[df['is_coffee_v3']]['ORDER_ID'].unique()
coffee_order_items = df[df['ORDER_ID'].isin(coffee_orders) & ~df['is_coffee_v3']]

# Group and count items by category
category_counts = coffee_order_items.groupby('Category')['ITEM_NAME'].value_counts().unstack(fill_value=0)
category_counts
## these aren't perfect, but good enough for this analysis


ITEM_NAME,"""Chicken"" Sandwich",1 Gallon Orange Juice,1-Topping Pizza Slice,10 Cheese Sticks,"10"" Gluten-Free Pizza",10-Pack Oatmeal Raisin Chocolate Chip Cookies,"12"" Buffalo Cauliflower, Scallion & Blue Cheese Pizza","12"" Butternut Squash, Ricotta & Cranberry Pizza","12"" Cheese Pizza","12"" Chicken & Basil Pizza",...,"Yogurt, Berries & Granola",Yuca Fries,Yuca w/ Pickled Red Onion,Za'atar Chips,Za'atar M'anoush,Zaika Chicken Curry,Zaika Lamb Curry,Zen Veggie Burger,Zesty Chicken Panini Boxed Lunch,Ziti w/ Tomato-Basil Sauce
Category,,,,,,,,,,,,,,,,,,,,,
Breakfast,0,0,0,0,0,1,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
Lunch,39,0,2,0,1,0,2,4,7,7,...,0,0,0,0,0,0,0,9,0,0
Other,0,2,0,1,0,0,0,0,0,0,...,0,1,1,2,8,6,2,0,1,1


In [171]:
# let's focus just on BREAKFAST category for this
coffee_breakfast_items = coffee_order_items[coffee_order_items['Category']=='Breakfast'].copy()

# exclude the coffee items themselves to focus on pairings
pairings = coffee_breakfast_items[~coffee_breakfast_items['is_coffee']]

# count the frequency of each item paired with coffee
pairing_counts = pairings['ITEM_NAME'].value_counts().reset_index()
pairing_counts.columns = ['Item', 'Frequency']

# display the top pairings
top_pairings = pairing_counts.head(25)
top_pairings

,Item,Frequency
0,Berry Smoothie,42
1,Full Sized Breakfast Burrito,18
2,Kale Crush Smoothie,16
3,Tropical Sun Smoothie,15
4,Fresh D-Tox Smoothie,13
5,Mind Booster Smoothie,11
6,Classic Breakfast Box,10
7,Power JP Smoothie,9
8,Bagels & Pastries,8
9,Breakfast Pastry Tray,8


This approach has given overall good results with some likely false-positive results (ex. Eggplant Meatballs), and more importantly establishes a framework that could be used to solve this problem with more accurate item- , customer-, or meal-information.

In this situation, we have the same problem before we challenges with using inclusion- and exclusion-lists as the basis for labeling (category here vs target earlier). With more accurate Category/meal labels, this analysis would be improved.